# Document Data Extraction Table and Text

In [ ]:
!pip install unstructured[all-docs] pydantic

In [ ]:
!sudo apt-get install poppler-utils tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.4).
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (4,354 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so 

In [ ]:
from unstructured.partition.pdf import partition_pdf
import os
from pydantic import BaseModel
from typing import Any

In [ ]:
dir_path = "/content/drive/MyDrive/Alemeno/pdf"
os.listdir(dir_path)

['goog-10-k-2023 (1)_removed_removed.pdf']

In [ ]:
documents = []
# raw_pdf_elements
for filename in os.listdir(dir_path):
    if filename.endswith(".pdf"):
        doc_path = os.path.join(dir_path, filename)

        raw_pdf_elements = partition_pdf(filename=doc_path,
                                        extract_images_in_pdf=False,
                                        infer_table_structure=True,
                                        chunking_strategy="by_title",
                                        max_characters=4000,
                                        new_after_n_chars=3800,
                                        combine_text_under_n_chars=2000,
                                        )

        class Document(BaseModel):
            page_content: Any
            metadata: dict

        count = 0
        for element in raw_pdf_elements:
            if "unstructured.documents.elements.Table" in str(type(element)):
                documents.append(Document(page_content=str(element), metadata={"source": filename, "page": count, "type": "table"}))
                count += 1
            elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
                documents.append(Document(page_content=str(element), metadata={"source": filename, "page": count, "type": "text"}))
                count += 1

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
documents

[Document(page_content='Table of Contents\n\nAlphabet Inc.\n\nNote 9. Goodwill\n\nChanges in the carrying amount of goodwill for the years ended December 31, 2022 and 2023 were as follows (in millions):', metadata={'source': 'goog-10-k-2023 (1)_removed_removed.pdf', 'page': 0, 'type': 'text'}),
 Document(page_content='Google Services Google Cloud Other Bets Total Balance as of December 31, 2021 $ 19,826 $ 2,337 $ 793 $ 22,956 Acquisitions 1,176 4,876 119 6,171 Foreign currency translation and other adjustments (155) (8) (4) Balance as of December 31, 2022 20,847 7,205 908 28,960 Acquisitions 240 3 0 243 Foreign currency translation and other adjustments 31 (9) (27) Balance as of December 31, 2023 $ 21,118 $ 7,199 $ 881 $ 29,198', metadata={'source': 'goog-10-k-2023 (1)_removed_removed.pdf', 'page': 1, 'type': 'table'}),
 Document(page_content='(167)\n\n(5)\n\nNote 10. Commitments and Contingencies\n\nCommitments\n\nWe have content licensing agreements with future fixed or minimum guara

In [ ]:
len(documents)

5

# Document Data Processing

Split into small chunks

In [ ]:
!pip install langchain-text-splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 8.3 MB/s eta 0:00:00


In [ ]:
from langchain_text_splitters import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [ ]:
docs

[Document(page_content='Table of Contents\n\nAlphabet Inc.\n\nNote 9. Goodwill\n\nChanges in the carrying amount of goodwill for the years ended December 31, 2022 and 2023 were as follows (in millions):', metadata={'source': 'goog-10-k-2023 (1)_removed_removed.pdf', 'page': 0, 'type': 'text'}),
 Document(page_content='Google Services Google Cloud Other Bets Total Balance as of December 31, 2021 $ 19,826 $ 2,337 $ 793 $ 22,956 Acquisitions 1,176 4,876 119 6,171 Foreign currency translation and other adjustments (155) (8) (4) Balance as of December 31, 2022 20,847 7,205 908 28,960 Acquisitions 240 3 0 243 Foreign currency translation and other adjustments 31 (9) (27) Balance as of December 31, 2023 $ 21,118 $ 7,199 $ 881 $ 29,198', metadata={'source': 'goog-10-k-2023 (1)_removed_removed.pdf', 'page': 1, 'type': 'table'}),
 Document(page_content='(167)\n\n(5)\n\nNote 10. Commitments and Contingencies\n\nCommitments\n\nWe have content licensing agreements with future fixed or minimum guara

In [ ]:
len(docs)

11

Make embedding function for store data into database

In [ ]:
import requests

model_id = "sentence-transformers/all-MiniLM-L6-v2"
hf_token = "enter_you_hugging_face_tocken"

api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
headers = {"Authorization": f"Bearer {hf_token}"}

def generate_embeddings(texts):
    response = requests.post(api_url, headers=headers, json={"inputs": texts, "options":{"wait_for_model":True}})
    return response.json()

In [ ]:
import uuid

embedded_doc = []
for doc in docs:
  id = str(uuid.uuid1())
  embedding = generate_embeddings(doc.page_content)
  meta = doc.metadata
  meta["page_content"] = doc.page_content
  meta

  embedded_doc.append({
      "id": id,
      "values": embedding,
      "metadata": meta
  })

In [ ]:
len(embedded_doc)

11

In [ ]:
len(embedded_doc[0]["values"])

384

Store the embedded data into pinecone

In [ ]:
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 4.3 MB/s eta 0:00:00


In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key="Enter_pinecone_tocken")
index = pc.Index("pdfvectorstore")

index.upsert(vectors=embedded_doc)

{'upserted_count': 11}

# Retreating an query from database

In [83]:
!pip install -qU langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 3.1 MB/s eta 0:00:00


In [111]:
# make your chatGroq key from https://console.groq.com/playground
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key="Enter_groq_api_query",
             model_name="Llama3-8b-8192")

In [107]:
def retreating_answer (query):
  embedded_query = generate_embeddings(query)
  query_response = index.query(
    top_k=3,
    vector=embedded_query,
    include_values=False,  # This will return the complete vector for each match
    include_metadata=True
  )
  db_output = query_response

  # print(db_output)

  description = ""
  for i in db_output['matches']:
    description += i['metadata']['page_content'] + "\n\n"

  # print(description)

  return description + query

In [108]:
llm_input = retreating_answer("What is the Diluted net income per share for class A")

Exception ignored in: <coroutine object retreating_answer at 0x7de9234a86d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py", line 27, in run
KeyError: '__builtins__'
Exception ignored in: <coroutine object retreating_answer at 0x7de9234a86d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py", line 27, in run
KeyError: '__builtins__'


In [ ]:
llm_input

In [119]:
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key="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",
             model_name="Llama3-8b-8192")

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([("human", "Answer the question{topic}")])
chain = prompt | llm
answer = await chain.ainvoke({"topic": llm_input})
print(answer.content)

According to the table, the diluted net income per share for Class A is $5.80.
